In [ ]:
%md
### Kardiaflow - Silver Feedback Enriched

**Source:** Snapshot join of Silver feedback with current Silver providers

**Target:** `kardia_silver.silver_feedback_enriched`

**Pattern:** LEFT JOIN from feedback to SCD2 providers, keeping only `is_current = TRUE` row

**Trigger:** CTAS; rerun after any update to feedback or providers

Notes: LEFT JOIN preserves all feedback rows, even if provider record is missing.

In [ ]:
from kflow.notebook_utils import init

init()

In [0]:
%sql
-- 1. Ensure Silver DB exists
CREATE DATABASE IF NOT EXISTS kardia_silver;
USE kardia_silver;

In [0]:
%sql
-- 2. Join feedback to current provider attributes (no CTE – inline subquery)
CREATE OR REPLACE TABLE silver_feedback_enriched AS
WITH current_providers AS (
  SELECT provider_id,
         provider_specialty,
         provider_location
  FROM   kardia_silver.silver_providers
  WHERE  is_current = TRUE
)
SELECT
    f.feedback_id,
    f.provider_id,
    cp.provider_specialty,
    cp.provider_location,
    f.timestamp,
    f.timestamp_source,
    f.visit_id,
    f.satisfaction_score,
    f.source,
    f.tags,
    f.metadata,
    f._ingest_ts,
    f._source_file,
    f._batch_id,
    CASE WHEN cp.provider_id IS NULL THEN true ELSE false END AS provider_missing,
    CASE WHEN e.encounter_id IS NULL THEN true ELSE false END AS encounter_missing
FROM kardia_silver.silver_feedback f
LEFT JOIN current_providers cp
  ON f.provider_id = cp.provider_id
LEFT JOIN kardia_silver.silver_encounters e
  ON f.visit_id = e.encounter_id;

In [ ]:
-- 3) Orphan feedback (missing encounter or provider)

WITH provider_keys AS (
    SELECT DISTINCT provider_id
    FROM kardia_silver.silver_providers
)
SELECT SUM(CASE WHEN e.encounter_id IS NULL THEN 1 ELSE 0 END) AS missing_encounter,
       SUM(CASE WHEN pk.provider_id IS NULL THEN 1 ELSE 0 END) AS missing_provider
FROM kardia_silver.silver_feedback f
LEFT JOIN kardia_silver.silver_encounters e
       ON f.visit_id = e.encounter_id
LEFT JOIN provider_keys pk
       ON f.provider_id = pk.provider_id;

In [0]:
%sql
-- 3. Preview: Most recent 5 feedback records
SELECT *
FROM kardia_silver.silver_feedback_enriched
ORDER BY timestamp DESC
LIMIT 5;